In [1]:
import os
import numpy as np
import pandas as pd
from mediapipe import solutions

from video_converter import Video2DataFrame
from custom_pose_landmarks import CustomPoseLandmark

In [ ]:
# # Prepare paths
# data_path = '../data/'
# video_path = os.path.join(data_path, 'raw/squat')

# # Read personal data from excel
# personal_data = pd.read_excel(os.path.join(data_path, 'PersonalData.xlsx'))

# # Processed data path
# processed_data = os.path.join(data_path, 'processed')

In [2]:
# Selected values of pose landmarks corresponding to PoseLandmark class from MediaPipe library
values = [0, 11, 12, 13, 14, 15, 16, 19, 20, 23, 24, 25, 26, 27, 28, 31, 32]

# Custom pose landmark names and their connections
landmarks = {
    'THORAX': ['NOSE'],
    'PELVIS': ['LEFT_HIP', 'RIGHT_HIP'],}

# MediaPipe solutions
mp_drawing = solutions.drawing_utils
mp_pose = solutions.pose

custom_pose = CustomPoseLandmark(mp_pose, values, landmarks)
conv = Video2DataFrame(mp_pose, mp_drawing, custom_pose)

In [3]:
len(custom_pose)

19

In [5]:
custom_pose.num_elements()

19

In [7]:
list(custom_pose.get_connections())

[(17, 0),
 (4, 6),
 (14, 16),
 (5, 7),
 (9, 11),
 (1, 3),
 (10, 12),
 (1, 9),
 (18, 10),
 (6, 8),
 (9, 10),
 (2, 4),
 (1, 2),
 (11, 13),
 (2, 10),
 (18, 9),
 (3, 5),
 (12, 14),
 (13, 15)]

In [48]:
class Graph():
    """
    
    """
    def __init__(self, layout, strategy='lol', max_distance=1, dilation=1):
        # Init
        self.max_distance = max_distance
        self.dilation = dilation

        self.edges = self.get_edges(layout)[0]


    def get_edges(self, layout):
        """
        
        """
        # Extract the number of nodes in the layout
        self.num_node = layout.num_elements()
        # Create a self-loop for each node
        self_link = [(i, i) for i in range(self.num_node)]
        # Extract connections from custome layout
        neighbor_link = list(layout.get_connections())

        # Prepare the edges and center of the graph
        edges = self_link + neighbor_link
        center = 1

        return edges, center


    def get_hop_distance(self):
        """ 
        
        """
        # Create a basic adjacency matrix
        adjacency_matrix = np.zeros((self.num_node, self.num_node))

        for i, j in self.edges:
            adjacency_matrix[i, j] = 1
            adjacency_matrix[j, i] = 1

        # Prepare a hop distance matrix storage
        distance_matrix = np.zeros_like(adjacency_matrix) + np.inf

        # Calculate the consecutive powers of the adjacency matrix
        transfer_matrix = [
            np.linalg.matrix_power(adjacency_matrix, d) for d in range(self.max_distance + 1)
        ]

        arrive_matrix = (np.stack(transfer_matrix) > 0)

        # Calculate hop distances for each node
        for d in range(self.max_distance, -1, -1):
            distance_matrix[arrive_matrix[d]] = d

        return distance_matrix


    def normalize(self, matrix):
        """ 
        normalize undirected graph, symmetry matrix
        """
        # Calculate degree (number of links) for each node
        degree_vector = np.sum(matrix, 0)

        # Prepare a diagonal matrix storage
        diagonal = np.zeros((self.num_node, self.num_node))
        
        # Calculate diagonal matrix
        for i in range(self.num_node):
            if degree_vector[i] > 0:
                diagonal[i, i] = degree_vector[i] ** (-0.5)
    
        normalized_matrix = np.dot(np.dot(diagonal, matrix), diagonal)

        return normalized_matrix
    

    def get_adjacency_matrix(self, strategy):
        """ 
        
        """
        hop_distances = range(0, self.max_distance + 1, self.dilation)
        adjacency_matrix = np.zeros((self.num_node, self.num_node))

        #
        distance_matrix = self.get_hop_distance()

        for hop in hop_distances:
            adjacency_matrix[distance_matrix == hop] = 1

        # normalize adjacency matrix
        normalized_adjacency = self.normalize(adjacency_matrix)

        if strategy == 'uniform':
            adjacency = np.zeros((1, self.num_node, self.num_node))
            adjacency[0] = normalized_adjacency

            return adjacency
        
        elif strategy == 'distance':
            pass

        elif strategy == 'spatial':
            pass

        else:
            raise ValueError('strategy error')

In [49]:
graph = Graph(custom_pose)

In [61]:
graph.num_node

19

In [64]:
hop_distances = range(0, 1 + 1, 1)
adjacency_matrix = np.zeros((graph.num_node, graph.num_node))

#
distance_matrix = graph.get_hop_distance()

for hop in hop_distances:
    adjacency_matrix[distance_matrix == hop] = 1

# normalize adjacency matrix
normalized_adjacency = graph.normalize(adjacency_matrix)

In [67]:
np.sum(normalized_adjacency, 1)

array([1.        , 1.01228193, 1.01228193, 0.9553418 , 0.9553418 ,
       1.07491496, 1.07491496, 0.90824829, 0.90824829, 1.14000458,
       1.14000458, 0.92486556, 0.92486556, 1.07491496, 1.07491496,
       0.90824829, 0.90824829, 1.        , 0.84973111])

In [27]:
for hop in hop_distances:
    print(hop)
    

0
1


In [28]:
distance_matrix

array([[1, 1, 0, 0],
       [1, 1, 0, 1],
       [0, 0, 1, 0],
       [0, 1, 0, 1]])

In [29]:
distance_matrix == 0

array([[False, False,  True,  True],
       [False, False,  True, False],
       [ True,  True, False,  True],
       [ True, False,  True, False]])

In [30]:
adjacency_matrix

array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])